In [1]:
# Prepare original change log 
import requests
version_major='5'
version_minor='10'
version_patch='27'
url = 'https://cdn.kernel.org/pub/linux/kernel/v'+version_major+'.x/ChangeLog-'+version_major+'.'+version_minor+'.'+version_patch
r = requests.get(url, allow_redirects=True)
ChangeLog='ChangeLog.txt'
open(ChangeLog, 'wb').write(r.content)

297052

In [2]:
# Extract Email addresses
import pandas as pd
pd.set_option('max_colwidth', None)
import re

f = open("ChangeLog.txt", "r")
emails = []
for line in f: 
    x = re.findall("<.*@.*>", line) 
    emails += x
len(emails)


emails_series = pd.Series(emails)

d = {
    "email": emails_series
}

df = pd.DataFrame(d)
df

,email
0,<gregkh@linuxfoundation.org>
1,<a.rabusov@tum.de>
2,<f.fainelli@gmail.com>
3,<linux@roeck-us.net>
4,<jason@bluehome.net>
...,...
1227,<stable@vger.kernel.org>
1228,<akpm@linux-foundation.org>
1229,<torvalds@linux-foundation.org>
1230,<hughd@google.com>


In [3]:
# Extract email suffix into new column

def extract_suffix(email):
    return re.findall('@.*>', email)[0]

df['email_suffix'] = df.apply(lambda x: extract_suffix(x['email']), axis=1)

df

# Exclude linux internal emails
internal_email_suffixes = ['@kernel.org>', '@linuxfoundation.org>', '@linux-foundation.org>']

index_to_exclude = df[ df['email_suffix'].isin(internal_email_suffixes)  ].index
df.drop(index_to_exclude, inplace = True)


In [4]:

# Group by email_suffix
df = df.groupby('email_suffix').size().reset_index(name='count')


In [5]:
# Sort by count

df = df.sort_values(by=['count'])

In [8]:
# add percentage column
count_sum = df['count'].sum()
print(count_sum)

df['percentage'] = df['count'] / count_sum
# print(df.to_string())

923


In [9]:
# get '@huawei.com>' row
df[df['email_suffix'] == '@huawei.com>']

,email_suffix,count,percentage
46,@huawei.com>,22,0.023835
